In [26]:
import json
import numpy
import pandas
from datetime import datetime
from pprint import pprint
import re
pandas.options.display.max_colwidth = -1
print('done')

done


In [27]:
# runs in about 5 minutes
#with open('events.json','r') as f:
with open('events_2012-2016.json','r') as f:
#with open('events_1927_1938.json','r') as f:
    allgames = json.load(f)
print('ok')
#pprint(allgames[2])   
allgames_bk = allgames

allgames_df = pandas.DataFrame(allgames)
allgames_df = allgames_df.set_index('id')
print('done')

ok
done


In [28]:
#mygames = ['NY1193809221']

#mygames = ['BAL201208060']
mygames = ['SEA201208150']
#mygames += ['BAL201208300']
#mygames += ['BAL201209300']


#mygames += ['BAL201304210']
#mygames += ['BAL201305170', 'BAL201405010', 'BAL201406130', 'BAL201406250', 'MIA201407030', 'PIT201605180']
#mygames += ['SDN201606150', 'WAS201607200', 'BAL201608180']

#weird_game_ids = ['CLE192708190', 'BOS193008310', 'BOS193107250', 'CHA193304220', 'NYA193606030', 'MLN195505152']
#weird_game_ids += ['MIN196907170', 'NYN197208160', 'KCA201109150', 'TOR201208130', 'KCA201605310']

somecolumns = allgames_df.columns
allgames2_df = pandas.DataFrame(columns=somecolumns)
allgames2_df['id'] = ''
allgames2_df = allgames2_df.set_index('id')
allgames2_df
for thisid, thisrow in allgames_df.iterrows():
    if (thisid in mygames):
        allgames2_df = allgames2_df.append(thisrow)
allgames_df = allgames2_df
print('done')

done


In [29]:
column_list = ['event_in_game', 'event_type', 'inning', 'home_or_visitor', 'batterID']
column_list += ['pitch_count', 'pitch_sequence', 'event', 'comment', 'sub_team', 'sub_batting_order', 'sub_position']

player_columns = ['playerID', 'name', 'batting_order', 'position']

display_order = ['gameID', 'visiting_team', 'inning', 'batting_team']
display_order += ['outs', 'balls', 'strikes', 'pitch_sequence']
display_order += ['visitor_score', 'home_score', 'batterID', 'batter_hand', 'pitcherID', 'pitcher_hand']
#display_order += ['catcherID', '1bID', '2bID', '3bID', 'ssID', 'lfID', 'cfID', 'rfID']
#display_order += ['runner1b', 'runner2b', 'runner3b']
print('ok')

ok


In [30]:
def load_rosters():
    all_rosters_df = pandas.read_csv('fullroster.csv')
    all_rosters_df['year'] = pandas.to_numeric(all_rosters_df['year'], errors='coerce')
    return all_rosters_df

def get_game_info(onerow, theindex):

    info_df = pandas.DataFrame.from_records(onerow['info'], index=theindex)

    teams_df = pandas.read_csv('teams.csv')
       
    info_df['visleague'] = teams_df['league'][teams_df['teamID'] == info_df['visteam'].tolist()[0]].tolist()[0][0:1]
    info_df['homeleague'] = teams_df['league'][teams_df['teamID'] == info_df['hometeam'].tolist()[0]].tolist()[0][0:1]

    info_df['attendance'] = pandas.to_numeric(info_df['attendance'], errors='coerce')
    info_df['date'] = pandas.to_datetime(info_df['date'], errors='coerce')
    info_df['dayofweek'] = info_df['date'].apply(lambda x: datetime.strftime(x, '%a'))
    
    info_df['gameno'] = pandas.to_numeric(info_df['number'], errors='coerce')
    info_df = info_df.drop('number', axis=1)
    info_df['starttime'] = pandas.to_datetime(info_df['starttime'], format='%l%M%P', errors='coerce')
    info_df['temp'] = pandas.to_numeric(info_df['temp'], errors='coerce')
    info_df['timeofgame'] = pandas.to_numeric(info_df['timeofgame'], errors='coerce')
    info_df = info_df.fillna('')

    return info_df


def get_starters(onerow):
    vsdf = pandas.DataFrame(onerow['visiting_starters'], columns=player_columns)
    vsdf['which_team'] = 0
    hsdf = pandas.DataFrame(onerow['home_starters'], columns=player_columns)
    hsdf['which_team'] = 1
    df = pandas.concat((vsdf, hsdf))
    df['batting_order'] = pandas.to_numeric(df['batting_order'])
    df['position'] = pandas.to_numeric(df['position'])

    return df


def get_inning_info(thisevent, infodf, inningno):
    
    thisevent['visiting_team'] = infodf['visteam'].tolist()[0]
    thisevent['home_team'] = infodf['hometeam'].tolist()[0]
    thisevent['inning'] = inningno
    if (side == 0):
        thisevent['batting_team'] = thisevent['visiting_team']
        thisevent['fielding_team'] = thisevent['home_team']
    else:
        thisevent['batting_team'] = thisevent['home_team']
        thisevent['fielding_team'] = thisevent['visiting_team']
    return thisevent

   
def get_fielders(thisevent, playing_df, fielding_flag):
                    
    thisevent['pitcherID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 1)].tolist()[0]                               

    thisevent['pitcher_hand'] = roster_df['throwing_hand'][
            (roster_df['playerID'] == thisevent['pitcherID']) & 
            (roster_df['year'] == float(thisevent['gameID'][3:7])) 
            ].tolist()[0]
    
    thisevent['catcherID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 2)].tolist()[0]
    thisevent['1bID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 3)].tolist()[0]
    thisevent['2bID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 4)].tolist()[0]
    thisevent['3bID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 5)].tolist()[0]
    thisevent['ssID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 6)].tolist()[0]
    thisevent['lfID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 7)].tolist()[0]
    thisevent['cfID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 8)].tolist()[0]
    thisevent['rfID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 9)].tolist()[0]

    return thisevent


def split_event(df):
    df['play'] = df['event'].str.split('/').str.get(0)
    df['modifiers'] = df['event'].str.split('/',n=1).str.get(1)
    df['play'] = df['event'].str.split('/').str.get(0)
    df['modifiers'] = df['event'].str.split('/',n=1).str.get(1)
    df['modifiers'] = df['modifiers'].fillna('')
    df['baserunning'] = df['event'].str.split('.',n=1).str.get(1)
    df['baserunning'] = df['baserunning'].fillna('')
    df['modifiers'] = df['modifiers'].str.split('.').str.get(0)
    
    return df


def get_sub_name(players_curr, theroster, evnt, gameinfo):
    
    if (evnt['sub_team'] == 0):
        sub_team_name = gameinfo['visteam'].values[0]
    else:
        sub_team_name = gameinfo['hometeam'].values[0]
    yeer = gameinfo['date'].apply(lambda x: x.year).values[0]
    sub_name_first = theroster['nameFirst'].iloc[
            numpy.where(
                (theroster['playerID'] == evnt['batterID']) & 
                (theroster['year'] == yeer) &
                (theroster['team'] == sub_team_name)
            )
        ].values[0]
    sub_name_last = theroster['nameLast'].iloc[
            numpy.where(
                (theroster['playerID'] == evnt['batterID']) & 
                (theroster['year'] == yeer) &
                (theroster['team'] == sub_team_name)
            )
        ].values[0]
    sub_name = '"' + sub_name_first + ' ' + sub_name_last + '"'

    return sub_name

def make_substitution(players_curr, theroster, evnt, gameinfo, basers):
#    print('\n')
#    print('SUBSTITUTING...')
    if (evnt['home_or_visitor'] == evnt['sub_team']):
        
        if (evnt['sub_position'] == 12):
            print('pinch runner')
            old_runner = players_curr['playerID'].loc[
                (players_curr['which_team'] == evnt['sub_team']) & 
                (players_curr['batting_order'] == evnt['sub_batting_order'])
            ].values[0]
            new_runner = evnt['batterID']
#            print(old_runner, new_runner)
            for i in range(0,len(basers)):
                if (basers[i] == old_runner):
                    basers[i] = new_runner
#            print(basers)
        
        players_curr.loc[
            (players_curr['which_team'] == evnt['sub_team']) & 
            (players_curr['batting_order'] == evnt['sub_batting_order'])
            , 'playerID'] = evnt['batterID']

        players_curr.loc[
            (
                (players_curr['which_team'] == evnt['sub_team']) & 
                (players_curr['batting_order'] == evnt['sub_batting_order'])
                ), 'name'] = get_sub_name(players_curr, theroster, evnt, gameinfo)
        
    else:
        
        players_curr.loc[
            (
                (players_curr['which_team'] == evnt['sub_team']) & 
                (players_curr['position'] == evnt['sub_position'])
                ), 'playerID'] = evnt['batterID']
        
        players_curr.loc[
            (
                (players_curr['which_team'] == evnt['sub_team']) & 
                (players_curr['position'] == evnt['sub_position'])
                ), 'name'] = get_sub_name(players_curr, theroster, evnt, gameinfo)
#        print('New lineup:\n')
#    print(players_curr)
    return (players_curr, basers)

def advance_runners(evnt, basers, oots, scr, teem):   # If there is baserunning info provided, obey it
    
    # If there are multiple baserunning events, split them around ;
    x = evnt['baserunning'].split(';')

    success = []
    failure = []

    for i in range(0, len(x)):
        if ('-' in x[i]):
            success.append([x[i].split('-')[0], x[i].split('-')[1][0], x[i].split('-')[1][1:]])
        if ('X' in x[i]):
            failure.append([x[i].split('X')[0], x[i].split('X')[1][0], x[i].split('X')[1][1:]])

    for u in (success, failure):
        for j in range(0, len(u)):
            if (u[j][0] == 'B'):
                u[j][0] = 0
            if (u[j][1] == 'H'):
                u[j][1] = 4
            for k in range(0,2):
                u[j][k] = int(u[j][k])

#    print('Failure matrix: ', failure)
#    print('Success matrix: ', success)
    
    for y in range(0, len(failure)):
        oots = oots + 1
        basers[failure[y][0]-1] = ''
        if (failure[y][1] < 4):            # no need to clear a base if runner is out at home
            basers[failure[y][1]-1] = ''
        evnt['comment'] += failure[y][2]     

    for z in range(0, len(success)):
        if (success[z][1] == 4):
            scr[teem] = scr[teem] + 1
        elif (success[z][0] == 0):
            basers[success[z][1]-1] = evnt['batterID']
        else:
            basers[success[z][1]-1] = basers[success[z][0]-1]

        basers[success[z][0]-1] = ''   # regardless, clear the base he was originally on
        evnt['comment'] += success[z][2]
    
    return (evnt, basers, oots, scr)

def advance_batter(evnt, basers, oots, scr, teem):   
    
    print('\n')    
    print('Advancing runners, then batter: Inning {0:.0f}, side {1:.0f}'.format(evnt['inning'], evnt['home_or_visitor']))
    
    # After baserunners have been advanced as needed, advance the batter too.
    if (evnt['play'][0] in ('I', 'F', 'E')):
        if ('B' not in evnt['baserunning']):   # now advance the batter to first, unless he's explicitly out
            basers[0] = evnt['batterID']
            
    elif (evnt['play'[0] == 'W']):
        if (re.match(r'WP',evnt['play'])):
            print('Wild pitch!')
        elif ('B' not in evnt['baserunning']):
            basers[0] = evnt['batterID']
        else:
            pass
            
    elif (evnt['play'][0] == 'P'):
        print('Passed ball!')
        
    elif (evnt['play'][0] == 'S'):
        if (evnt['play'][1] == 'B'):   # a single as opposed to a stolen base
            print('Stolen base!')
            stolen_base = int(evnt['play'][2])
            orig_base = stolen_base - 1
            basers[stolen_base-1] = basers[orig_base-1]
            basers[orig_base-1] = ''
            print(basers)
            evnt['abflag'] = 0
        elif ('B' not in evnt['baserunning']):   # now advance the batter to first, unless he's explicitly out
            basers[0] = evnt['batterID']
        else:
            pass   # we already explicitly advanced the batter above
        
    elif (evnt['play'][0] == 'D'):
        if (evnt['play'][1] == 'I'):   # Defensive indifference, ignore
            evnt['abflag'] = 0
            pass
        elif ('B' in evnt['baserunning']):
            pass
        else:
            basers[1] = evnt['batterID']

    elif (evnt['play'][0] == 'T'):
        if ('B' not in evnt['baserunning']):
                basers[2] = evnt['batterID']

    elif (evnt['play'][0] == 'H'):
        if (evnt['play'][1] == 'R'):
            scr[teem] = scr[teem] + 1
            basers = ['', '', '']
        elif (evnt['play'][1] == 'P'):  # HP means hit by pitch
            basers[0] = evnt['batterID']
        else:
            print('H but not R or P')
    
    elif (evnt['play'][0] == 'C'):   # caught stealing
        print('Caught stealing!')
        stolen_base = int(evnt['play'][2])
        orig_base = stolen_base - 1
        oots = oots + 1
        basers[orig_base-1] = ''
        print(basers)    
        evnt['abflag'] = 0
        
    else:
        print('wtf starting letter not in set')

    print ('Now on base: [{0}, {1}, {2}] with {3} outs; score {4}-{5}'.format(basers[0], basers[1], basers[2], oots, scr[0], scr[1]))
    print('\n')
    
    return (evnt, basers, oots, scr)

print('done')

done


In [33]:
roster_df = load_rosters()

events_df = pandas.DataFrame()

for thisgameID,thisrow in allgames_df.iterrows():
    print(thisgameID+'...')
    game_events_df = pandas.DataFrame()
        
    thisgame_df = pandas.DataFrame(thisrow['events'], columns=column_list)
    thisgame_df = thisgame_df.fillna('')
    thisgame_df['gameID'] = thisgameID
    thisgame_df['inning'] = pandas.to_numeric(thisgame_df['inning'])
    thisgame_df['home_or_visitor'] = pandas.to_numeric(thisgame_df['home_or_visitor'])
    thisgame_df['sub_team'] = pandas.to_numeric(thisgame_df['sub_team'])
    thisgame_df['sub_batting_order'] = pandas.to_numeric(thisgame_df['sub_batting_order'])
    thisgame_df['sub_position'] = pandas.to_numeric(thisgame_df['sub_position'])
    
    
    thisgame_df = thisgame_df.set_index('event_in_game')
    
    this_info_df = get_game_info(thisrow, allgames_df.index) # get constants of game info
    playing_df = get_starters(thisrow) # get starters
    thisgame_df = split_event(thisgame_df) # split all events in game into parts    
    score = [0,0] # score as [vis, home] starts at [0,0]

    foundinnings = thisgame_df['inning'].drop_duplicates().values.tolist()
    theseinnings = numpy.arange(1, numpy.max(foundinnings)+2) # it's +2 b/c it expects a 0-based list

#    If we need to switch the sub side.
#    thisgame_df['home_or_visitor'] = numpy.where(thisgame_df['event_type'] == 'sub', 1 - thisgame_df['home_or_visitor'], thisgame_df['home_or_visitor'])
#    print('STARTERS:')
#    print(playing_df)

    for ing in theseinnings:
        
        for side in range(0,2):                           # iter through each half-inning
            halfinn = thisgame_df[(thisgame_df['inning'] == ing) & (thisgame_df['home_or_visitor'] == side)]
            onbase = ['', '', '']
            nOuts = 0            #     and there are no outs
            fieldingflag = 1 - side  # flag: makes fielding team the opposite of batting team

            for eig, thisevent in halfinn.iterrows():    # iter through all events in this half-inn    

                if (thisevent['event_type'] == 'sub'):
                    thisyear = this_info_df['date'].apply(lambda x: x.year)
                    (playing_df, onbase) = make_substitution(playing_df, roster_df, thisevent, this_info_df, onbase)
                else:
                    print(thisevent[['inning', 'home_or_visitor', 'batterID', 'event']].tolist())
                    thisevent['visiting_team'] = this_info_df['visteam'].tolist()[0]
                    thisevent['home_team'] = this_info_df['hometeam'].tolist()[0]
                    thisevent['inning'] = ing
                    if (side == 0):
                        thisevent['batting_team'] = thisevent['visiting_team']
                        thisevent['fielding_team'] = thisevent['home_team']
                    else:
                        thisevent['batting_team'] = thisevent['home_team']
                        thisevent['fielding_team'] = thisevent['visiting_team']

                    thisevent = get_fielders(thisevent, playing_df, fieldingflag) # get fielders, including pitcher

                    thisevent['abflag'] = 1
                    thisevent['balls'] = pandas.to_numeric(thisevent['pitch_count'][0:1], errors='coerce')
                    thisevent['strikes'] = pandas.to_numeric(thisevent['pitch_count'][1:2], errors='coerce')

                    # batterID gets loaded from raw data, but look up his handendess... and....
                    # DON'T FORGET TO CORRECT THIS FOR BADJ IF NEEDED!
                    thisevent['batter_hand'] = roster_df['batting_hand'][(roster_df['playerID'] == thisevent['batterID']) & (roster_df['team'] == thisevent['batting_team']) & (roster_df['year'] == float(thisevent['gameID'][3:7]))].tolist()[0]

#                    if (thisevent['play'] == 'NP'):
#                        print('\n')
#                        print('NO PLAY: ',eig, thisevent[['inning','home_or_visitor','batterID', 'event']].tolist())
#                        print('-------')

                    # if there is baserunning info, implement it first
                    if (thisevent['baserunning'] != ''):
                        (thisevent, onbase, nOuts, score) = advance_runners(
                            thisevent,
                            onbase,
                            nOuts,
                            score,
                           side)               

                    # If the play starts with fielder info or is a strikeout, increment outs
                    if (re.match(r'[0-9K]+',thisevent['play'])):
                        nOuts = nOuts + 1
                    else:
                    # If the play is a hit, advance baserunners with fcn advance_runners(evnt, basers, oots, scr, teem):
                        (thisevent, onbase, nOuts, score) = advance_batter(
                            thisevent,
                            onbase,
                            nOuts,
                            score,
                            side)

                    # update info based on the play that has occurred
                    thisevent['outs'] = nOuts
                    thisevent['runner1b'] = onbase[0]
                    thisevent['runner2b'] = onbase[1]
                    thisevent['runner3b'] = onbase[2]
                    thisevent['visitor_score'] = score[0]
                    thisevent['home_score'] = score[1]

                # finally, append this play to the list of plays
                game_events_df = game_events_df.append(thisevent)
                
    game_events_df = game_events_df[game_events_df['event_type'] == 'play']
    game_events_df = game_events_df.reset_index()
    game_events_df.index.name = 'event_in_game'
    
    events_df = pandas.concat((events_df, game_events_df), axis=0)

#display_order = ['gameID', 'event_in_game', 'event_type', 'inning', 'batterID']
display_order = ['inning', 'batterID']
display_order += ['play', 'modifiers', 'baserunning', 'outs']
display_order += ['runner1b', 'runner2b', 'runner3b', 'visitor_score', 'home_score']

#events_df = events_df.fillna('')

events_df = events_df.reset_index()

events_df['event_in_game'] = events_df['event_in_game'] + 1  # make it a 1-based counter
events_df.index.name = 'eventID'

events_df.sort_values('gameID')
events_df = events_df[display_order]
print('Final score: ',score)
#events_df[events_df['event_type'] == 'play']
events_df

SEA201208150...
[1, 0, 'fulds001', '9/F']
[1, 0, 'uptob001', '63/G']
[1, 0, 'joycm001', '43/G']
[1, 1, 'ackld001', 'S7/G']


Advancing runners, then batter: Inning 1, side 1
Now on base: [ackld001, , ] with 0 outs; score 0-0


[1, 1, 'saunm001', '4/LDP.1X1(43)']
[1, 1, 'montj003', '5/L']
[2, 0, 'longe001', 'K']
[2, 0, 'zobrb001', '63/G']
[2, 0, 'penac001', '7/F']
[2, 1, 'jasoj001', '63/G']
[2, 1, 'seagk001', '7/F']
[2, 1, 'smoaj001', 'E6/G']


Advancing runners, then batter: Inning 2, side 1
Now on base: [smoaj001, , ] with 2 outs; score 0-0


[2, 1, 'robit001', 'S7/L.1-2']


Advancing runners, then batter: Inning 2, side 1
Now on base: [robit001, smoaj001, ] with 2 outs; score 0-0


[2, 1, 'thame001', '7/F']
[3, 0, 'lobaj001', '9/F']
[3, 0, 'johne002', 'K23']
[3, 0, 'rodrs002', '7/F']
[3, 1, 'ryanb002', 'S7/L']


Advancing runners, then batter: Inning 3, side 1
Now on base: [ryanb002, , ] with 0 outs; score 0-0


[3, 1, 'ackld001', '6/P']
[3, 1, 'saunm001', '7/L']
[3, 1, 'montj003', '

,inning,batterID,play,modifiers,baserunning,outs,runner1b,runner2b,runner3b,visitor_score,home_score
eventID,,,,,,,,,,,
0,1.0,fulds001,9,F,,1.0,,,,0.0,0.0
1,1.0,uptob001,63,G,,2.0,,,,0.0,0.0
2,1.0,joycm001,43,G,,3.0,,,,0.0,0.0
3,1.0,ackld001,S7,G,,0.0,ackld001,,,0.0,0.0
4,1.0,saunm001,4,LDP,1X1(43),2.0,,,,0.0,0.0
5,1.0,montj003,5,L,,3.0,,,,0.0,0.0
6,2.0,longe001,K,,,1.0,,,,0.0,0.0
7,2.0,zobrb001,63,G,,2.0,,,,0.0,0.0
8,2.0,penac001,7,F,,3.0,,,,0.0,0.0
